In [2]:
from sage.all import *
import struct

DR = RealField(53)

DD = RealField(190)

def double_to_hex(f):
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x = RealField(190)(x).exact_rational()
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def split_triple_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_mid = DR(x - DD(x_hi))
    x_lo = x - DD(x_hi) - DD(x_mid)
    return (x_lo, x_mid, x_hi)

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")

def print_triple_double(mark, x):
    splat = split_triple_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}, {double_to_hex(splat[2])}),")

In [3]:
from sage.all import *
from mpmath import mp, besseli, taylor

# High precision
mp.prec = 107

# Step 1: Define I0(x) - 1
def shifted_i0(x):
    return besseli(0, x) - mp.mpf(1)

# Step 2: Series of I0(x) - 1 in terms of (x/2)^2
terms = 100
from mpmath import taylor
coeffs = taylor(shifted_i0, 0, terms)

# print(coeffs)

# Step 3: Build series in terms of y = (x/2)^2
R = PolynomialRing(RealField(107), 'y')
y = R.gen()
f = R(0)

for n in range(2, terms, 2):
    k = n // 2
    c = RealField(107)(coeffs[n])
    if n >= 1:
        f += R(c) * y**(k-1) * (4**k)
    else:
        f += R(c) * y**(k-1) * (4**k)

print(f)

rat_approx = SR(f).power_series(RealField(107)).pade(28, 0)
numer = rat_approx.numerator()
denom = rat_approx.denominator()

# print(numer)
# print(denom)

def I0_approx(y):
    z = (y/2)**2
    return 1 + z * numer(z) / denom(z)

# Print result
print("Rational approximation of I0(x):")
# pretty_print(I0_approx)

# Evaluate at some point
print("I0_approx(1) =", I0_approx(-8.5))
print("I0_real(1) =", besseli(0, -0.5))

print("Numerator polynomial coefficients:")

coeffs = numer.coefficients(sparse=False) 

for i, c in enumerate(coeffs):
    print("")
    # print_double_double("")
    # print("f64::from_bits(" + double_to_hex(c) + "),")

2.702652841584581112466391783809e-126*y^48 + 6.489069472644579251031806672925e-123*y^47 + 1.495081606497311059437728257442e-119*y^46 + 3.302635268752560130297941720689e-116*y^45 + 6.988376228680417235710444680978e-113*y^44 + 1.415146186307784490231365047898e-109*y^43 + 2.739723016691870773087922732730e-106*y^42 + 5.065747857863269059439569132818e-103*y^41 + 8.935979221270806620851399950291e-100*y^40 + 1.502138107095622592965120331644e-96*y^39 + 2.403420971352996148744192530631e-93*y^38 + 3.655603297427907142239916839089e-90*y^37 + 5.278691161485897913394439915644e-87*y^36 + 7.226528200074194243436988244516e-84*y^35 + 9.365580547296155739494336764894e-81*y^34 + 1.147283617043779078088056253700e-77*y^33 + 1.326259861302608614269793029277e-74*y^32 + 1.444296988958540780939804608882e-71*y^31 + 1.478960116693545759682359919496e-68*y^30 + 1.421280672142497475054747882636e-65*y^29 + 1.279152604928247727549273094372e-62*y^28 + 1.075767340744656338868938672367e-59*y^27 + 8.434015951438105696732

In [109]:
from mpmath import mp, besseli

# Setup precision
mp.dps = 100

# Define the interval for y (example: from 0 to 3)
a = 0.0
b = 7.5

# Number of test points
N = 1000

# Use your RealField 300 for evaluation
RF = RealField(107)

# Prepare to store max relative error
max_rel_error = 0

for i in range(N+1):
    y_val = a + (b - a) * i / N
    y_rf = RF(y_val)
    
    approx_val = I0_approx(y_rf)
    true_val = mp.besseli(0, y_val)
    
    # Compute relative error
    rel_err = abs(approx_val - true_val) / abs(true_val)
    
    if rel_err > max_rel_error:
        max_rel_error = rel_err

print(f"Maximum relative error on [{a}, {b}] is about {-RealField(70)(max_rel_error).log2()}")

Maximum relative error on [0.0, 7.5] is about 104.35860052789506305


In [324]:
from mpmath import mp, besseli, nprint, taylor, chebyfit

# High precision
mp.prec = 1000

# Define the function to expand
def f(x):
    return mp.sqrt(x) * mp.exp(-x) * besseli(0, x)

def f_v(v):
    x = mp.mpf(1) / v
    return f(x)

# Generate coefficients of series in u = 1/x
terms = 80
coeffs = taylor(lambda u: f_v(mp.mpf(u)), mp.mpf('1e-70'), terms)

# print(coeffs)

def f_z(v):
    x = mp.mpf(1) / v
    return f(x)

# coeffs = chebyfit(lambda u: f_z(mp.mpf(u)), [1/9.5, 1/709], terms)
# coeffs = list(reversed(coeffs))
# print(cheby)

print(coeffs)

# for n in range(terms):
#     u = mp.mpf('1e-30')  # Very small u ≈ 1/x
#     deriv = mp.diff(lambda u: f(1/mp.mpf(u)), u, n)
#     coeff = deriv / mp.fac(n)
#     coeffs.append(coeff)
    # print(f"u^{n}: {coeff}")

x = var('x')
u = x
# shifted_poly = sum(RealField(120)(coeffs[k]) * (u - u0)**k for k in range(terms))
coeffs = [QQ(RealField(120)(coeffs[k]).exact_rational()) for k in range(terms)]
# print(shifted_poly)

# Construct polynomial approximation in 1/x
# P_ring = PolynomialRing(RR, 'u')
taylor_poly = sum(c * u**k for k, c in enumerate(coeffs))

# print(taylor_poly)

pade_approximant = SR(taylor_poly).power_series(RealField(120)).pade(26, 0)

expr_in_x = pade_approximant.subs(u = 1/x)

numer_asympt = expr_in_x.numerator()
denom_asympt = expr_in_x.denominator()

# print(numer_asympt)

# print(denom_asympt)

print(float(f(10.1)))
print(float(numer_asympt(RealField(53)(1/10.1))/denom_asympt(RealField(53)(1/10.1))))
print(taylor_poly(x=1/RealField(53)(1/10.1)))
print(expr_in_x(x=1/10.1))

coeffs = numer_asympt.coefficients(sparse=False) 

print("coeffs")

for i, c in enumerate(coeffs):
    # print_double_double("", c)
    print(RealField(53)(c))
    # print("f64::from_bits(" + double_to_hex(c) + "),")

[mpf('0.398942280401432677939946059934381868475858631164934657665925829670657930886080343519160808156632685610075914810868072247194212778628708126220566978189396229351120948630596058597774216847352010822213518130622312314298187726068248892921535066452649157368443697030420078110204867940007787097092290942139318'), mpf('0.0498677850501790847424932574917977335594823288956168322082407287088322463475385479578035752689048348810328452995913985925824974214014593990024861324193775663780935336357039728605238243458161042836901840112548707297066779172292843671610598281821272062813116683914436610483797405352556726930854268262900815'), mpf('0.0280506290907257351676524573391362251272088100037844681171354098987181441806162513714115446192504374478260009227821624652212782266203006555681728638157167530085960324034970534518418452833237352295512933461294495543422222622472631587812587150864734701120626594760097409615589279255986008315817046717414888'), mpf('0.029219405302839307466304643061600234507509177087

In [74]:
from mpmath import mp, besseli

# Setup precision
mp.prec = 150

# Define the interval for y (example: from 0 to 3)
a = 9.0
b = 13.0

# Number of test points
N = 1000

# Use your RealField 300 for evaluation
RF = RealField(107)

# Prepare to store max relative error
max_rel_error = 0

def I0_approx_asympt(y):
    return numer_asympt(x=1/y) / denom_asympt(x=1/y) * RealField(53)(y).exp() / RealField(53)(y).sqrt()

for i in range(N+1):
    y_val = a + (b - a) * i / N
    y_rf = RF(y_val)
    
    approx_val = I0_approx_asympt(y_rf)
    true_val = mp.besseli(0, y_val)
    
    # Compute relative error
    rel_err = abs(approx_val - true_val) / abs(true_val)
    
    if rel_err > max_rel_error:
        max_rel_error = rel_err

print(f"Maximum relative error on [{a}, {b}] is about {-RealField(107)(max_rel_error).log2()}")

Maximum relative error on [9.0, 13.0] is about 19.36698681799193211160351046461


In [19]:
from mpmath import mp, besseli, nprint

# High precision
mp.prec = 150
print(besseli(0, 16.225000000016742))

1110969.1973027737067063330973042313994430942


In [193]:
from mpmath import mp, besseli, chebyfit
from sage.all import *

# Set high precision in mpmath
mp.prec = 53

# Step 1: Define the function
def f(x):
    return mp.sqrt(x) * mp.exp(-x) * besseli(0, x)

coeffs = chebyfit(lambda u: f(mp.mpf(u)), [8, 709.782712893384], 27)
print(coeffs)

for i, c in enumerate(reversed(coeffs)):
    # print_double_double("", c)
    # print(c)
    print("f64::from_bits(" + double_to_hex(c) + "),")


[mpf('5.1033286636335007e-65'), mpf('-4.9448046485473206e-61'), mpf('2.2644503491559604e-57'), mpf('-6.5181879689808576e-54'), mpf('1.323100797534861e-50'), mpf('-2.0141049641424839e-47'), mpf('2.387233204663285e-44'), mpf('-2.2578753029007174e-41'), mpf('1.7327243128674407e-38'), mpf('-1.0912375418023538e-35'), mpf('5.6826614926496948e-33'), mpf('-2.4583026152180637e-30'), mpf('8.8538394385740462e-28'), mpf('-2.6551324549089779e-25'), mpf('6.6174596019044181e-23'), mpf('-1.3655130349836739e-20'), mpf('2.3193546998441735e-18'), mpf('-3.2166452699116028e-16'), mpf('3.6040093138654318e-14'), mpf('-3.217796031208781e-12'), mpf('2.249536548765481e-10'), mpf('-1.2039009184205734e-8'), mpf('4.789637519369099e-7'), mpf('-1.3621786829552683e-5'), mpf('0.00026227316601194486'), mpf('-0.0031505266377368203'), mpf('0.4194041232607285')]
f64::from_bits(0x3fdad784644d97aa),
f64::from_bits(0xbf69cf221bf9e8bb),
f64::from_bits(0x3f313036abaf0acd),
f64::from_bits(0xbeec912420536339),
f64::from_bits(0x3

In [12]:
values = [
'0.3989422804014326804709228373048015407846313361341041003988890351647436530718236121489230915761914908856511272621890463775329098234819866212877204319468339065589989281682495541039800175979358733962407289045148725150011510571944345386248690017713934190147614070842979040763922377292275849617949868710275',
'4.9867785050173889722376571108046455736204291935161996722943848985865733257614526590068580374735524881600728870953049181770949530564716823443529991110190227199487251156222229401653316279153769989082063065601249935456141396963022676907353478915006381367075669626238986542778976733455009638874697768728186e-2',
'2.8050629095111159343796109879193528372067895544353841268095796249671996377413567432053082097959086976198011174985312832634366762375794522708798130922063161008669461160519127151166794212369322306269174249667541589090079504300468004142225802467491644828246867454257163731293538044267713316248435270683063e-2',
'2.9219403265024467438319422324353394263095992378979122387687824528939678515025816811358438664222278938931406413624322719398512077836755141345211828389654999129657463672033139125356887822032205569034269146238899433320629460121223465741461282283662031806685623030457251354411558730253488084571347984528925e-2',
'4.4742804860264050646357831544466387641049218534656145835668407252204478326063537934386718244509938659008253390482646852665359361604594302638827385952112035274058952070647084880212251782531740836181542320504771830281919914126413323165758602335108112649205133551779682834162365804141907081432067394421655e-2',
'9.048844504877660079660522033180949054355142116109391384763984895898817716170580634463502841995457547807555045612894689270302641583287687633101849911685069872639833344905435670307538628501078972765711801708242513924431251667310573804814968152887465865139665374432671505408737965722693934220781323181974e-2',
'0.2439348033696209643982922809438206473514632573897956739642428555399410321598310093486432493732069099330952270545116975092343255016238268049520942590242045682544964565189475181963316421604414248064882217196631640318823449857941879768275302013151082679125177641226079810553139298563179409195524184755182',
'-0.8227405263113711833232333408040363931241742853037086606566075842013642952074588803860032733263593254446567133178053673195295329298814296559178093530132151540830922845292523012411892694687448711289006783734665018018898191609548706920932781176606591152261137984429163116936319885102736895375946310326737',
'108.54090244531241814378663561746325947349692910836425856265714718785261570314398365277355324596645992688143575199803187527206091971538995730680238791450135161187967262932232522712918009426328622566632069837197403337505425887547810587332111936618474356780163882510058917460881077483671827700837937469391',
'-5221.4910104610963554903755195285257334861006325138284269172369049248939592844261370569038198417211895478463703051727046034150598934570133649843386832865005065173331942722917347045522615754836142753181014887422667700215902514331935659260267160517377144775615405013498121960431842725594010450270321494445',
'1.5957894943881449590812413637797564394316523209647481459015053370930564358913521952455768020543215609872784135599674811728103511861574741841058363335176261529376777462215512001144998754467870238613710097549018651522674797026929625579243878131708877290355593063973651522104266484988588110490395493431865e5',
'-7.978216188534547954599456887498096638009087169968096421828723310988102342393950469026501055827452879125902155154061544398229504886852102271452769670365370306511769817274965573173648045065645612915220210680743170946096492555396325002063762161783613150373760959498067830837248507812174859228416415596227e5',
'-2.1993729752547558658918232506775760934256239764567955271375595943720183027756578443983766863826659031844047711347817990539067410174621791027785399241632504729869301916563901014388065156146394013831985664755114907331698686779325420153908285559761656253708215732336850967453688230149663555428149163175624e8',
'1.4537517017087537665712471591222048330503409097124635351720732744713748115893898172785322245190525468079635896570696880437275324938768014392058959126067932506788788160398798010672877531005667720907663590809566425240237775456185735486606029658560070503493422822973755754045571062539448277206015997848154e10',
'-5.66215985506028673796838142194079955489586301304979319320959280503300994552380188717590960071735755222739856938087907626439589157633518509033972484773324136272662988956583311066651986811560329445209809265037109718755459637419109908946107951060711268851677650362339431754022301818593192089933801436305e11',
'1.5975303656798134107899638175335080370260749196468881433837859323828270794390657470124520823577055821791733481188673342628845827216520471733813868669642139864536391707790584694605434451923911102645049610148601443905599902956225611636317182215269660690468799767707490604333954280516801027803529941080015e13',
'-3.4663759624261648011763183729279592832039023482922933374998449686137014913153200315081064535648217628600709233294969520278502036708437363639302443100974615512544532020815009524689155784092821245788119344382164113850320176472024829678763260660551244376726915193594143302638649207266914706355396383767306e14',
'5.923253664452235844290494090423849881442682420463185273325002480109626960036004593702914630081080588776697033009961226770931222308394837970378660138419784231221211370937671201931549401728678971578007539022113488327724942015592988148718715883119395764717648761374203642676165074259987544543283531748171e15',
'-8.042800104440255510096906473194715716745003815996028750741733255596130102208864092152652005413325415154370324840158843538413484690814784399953642508942773717158531624165296256092080108437592451022856881895091943631402386378828827993254093223452935949710924049767418356204448473281914093080847479906228e16',
'8.682662991104459578652612268258317342976819756992393462850613709936340253699315915982717442419941041574011704945421720821340419786554723288122957232387624727450937547121490156279461542243158518420076208857350051019904671038115596599115860023149262477425293485842397373098011299881313397003237141275364e17',
'-7.407486652841340639970312211920275959147714622840219312435583909480517315863365449487403602501086582641416702591674374078866678892561110032518705883408138828303847235642180775648497494486507330045263875635378978891311944695537390981349228551008507908854761071384004930964641350353596054360798163343625e18',
'4.9304024365260227702403711463194886125230972806893885765302685007767765080006986531676473786477193995407659244896761311216806441458349708353194611433943461626791215889822831830572463842418439315715322914946785996751898209339788973655155530769810309546310013779094370059253934282999045851509814007453244e19',
'-2.5055193898853453501499047787122901825445550893025750539638324827086331995699576325772752739797151359720586558001016963586102045146725593646321150556935207936033817218182587842841935231799667367023929452878892909533393631439707770912807494350305599446486576052181328067881957611640827488764692826906778e20',
'9.385763572143302868017418507446394694523492625446284122402468252723978501452332064627331464290548154820636231767630313253317399571197556458234324939044478090603392739883798103574364781587014124777314445730655253385976129713882632884351599175016548693466653566700276177155164898790224912648816529128996e20',
'-2.4421978477427978969072203429432456579576393378033451923458926183043135492099773816240168985574950545249973546066884751427142663286946902754663894443360867642830649605805575260323554509700705559716724632393145127707748779363218682312700493154153537706238712306264138357456222429620719970442329320586676e21',
'3.941618692547453078506027967196977553491549475394874818067061380375161221276726037761521353150491161349739089306820102257093033748416779107344419515001603774666130987714545490348321626881952575131761677195641565137748353923555768251285402454225184542678162506845773425531923666937082069701448544876675e21',
'-2.9718310912745987885416677399481976797251773360174014782901403965200197187620192073704232587592967747615903198294248693491144825789369190584218677508933097617574634344204944150472028124995858965158605495314196137760150166726127776939677518184432146780440117682539979405982227670226812087629189829518103e21',
]

values = [
' 0.3989422804040553195386896442116217240934917032432631121815693653080639720721702081947254382169191779308112504184540728812678349536414528409303184508312771533479119775548557463345628472287487123617079446750596352244740819555843706330342579406172310466229905920859410274012923134304457971616088290327502 ',
' -0.14960335674043679602004261738116893360571799503540514384212327159285493652214637153907135247746991449797374125084392386253623739866781980679246175169654888553715018622056294668783464117023796473367646792195501894159125664218143995238469408967361407934601810434699175193174574382663264163610318555009155 ',
' -4.675060685345813481437728514997955120358843414352374684304817453982466784581587472510278750700826508505406832285730842896441387950675341387064323659518143833321819743252899152473989091694720262541978218622124993294326706070719272625090914713964382540317869184438618172448198720306981736250924654966409e-2 ',
' -4.098193939238402537265736306456632684997784422790979649230604244399255210761975836323172331683217261233946147565695292413591868660700279538602443786165479054705411567382398749780129421989374238579183818522568269329421172668332497312212535194124520091911690753740349201957858054155555351331601905402333e-2 ',
' -4.8865106868944186315566578232640413946380650420525906847571918906928424148716480871375348968573818218252883729884489663069340311591302072522431676329072621207473024529436390430517064521381624281102771549623288841692173301338780197996636350672262235714644251697449403555402205497352832956301777411810566e-2 ',
' -0.84097748732806726256822162232273960124911215606146781473131944352126474165038441787967847280119547016014701836560789655316137180382005732538469691375336215542100203835125810324386191389311768811517627107253927336666154705161068092064567038306226503035605835165668500644016798726711718685748394462446915 ',
' 46.298572865372920552174086939838026069846316354672659122543093433802839044602241582925754561197545765599345766455134090100602465877005510626617341354202178756010127034604055728074803935296666839822567895712064444928717653419399413749738000926349620375334187579423379054256638857838011678797843063083065 ',
' -2304.37249691023668264212143290783886395676526321564467352056347394776427016713108124531471090346522353030632665168128261048335911513954604171713732317100354744067981699024986529133400708452357851121484950514227150930238467064315636992404814694520805061072383542735057425354957260832638626343025612157 ',
' 89928.69341769416707241108845556838236859614274155828188047577873008324391420548517096466645300911082271169574508134269577516382171755740537495024807502016391571581103195908026435357998613976199012386472882855782344770665237806625634534937566166964053543333553537312996232259168016072230217099495502307 ',
' -2.80399484026712756909325836934262641603552066093377309266726929955217837156085338838561729652192996374513638734072865612371782491204380432312042444749563616070598760389020381108663665741349848753425064006313246259540799868316483843953744515784859687251732994295382858069164787181043330826148855409959e6 ',
' 7.037205843621445174280861643755952480041377241042083650440454155548371918505039728550849586644139920024301958677959769316738853527624640202046668398096665887194780659507691602735041016268278611689697875463009294089497503828126940137577284921455845775531846977476425323490990293645128739946551579034733e7 ',
' -1.4281729475416455176068221988326182237797562797545965396892703825419674144117873115780255840876017857444353003336649396507324763135387841634171886013492432753041874743604816631997226344712995230538877959365567683126471461396965754989694529727101376046812430759009621634625172978735619804447329030556255e9 ',
' 2.3478912187964354170437449213477189058154403516566701978398962838690902730110808657756965551292030451954512051038893783968707437164252363057103920079775571081235502645571983765899079498565628845802180399344530860292316264472616403699083102299322566501422414190506270182069445488083489310684093437917505e10 ',
' -3.123846542363811657483924006285407580825721780105751145197201403567329065032734400019923385972908364358130299067090861622719221187355754270852638442545196655019107010400796582833235100025646195040626216920809230885642459064466329548902602829288414512603867946737560363636271504291328779093376528680377e11 ',
' 3.351331815544722303181002920528009701465211527736849075153397825557004862087836072908677520772595919844985402773148957833271254988684010181938461956912316851604569470147699496822303890107083659435773495601120112503470748513741111461572629862061722869217674827765737491197122164116068691549267921762524e12 ',
' -2.879531478453963114395147012211587619610651019873621444406531091928011524651000206145266130796698236829838960534950534706621860493832899480714629589021022034699220840427565220607415988656058121748903762609264593140992121214833877034600698839931144396588929788407209395327029036644144877734874145824588e13 ',
' 1.9606911508846312254560927730550162135996932196033266601906878537120301021467028233789085559499656678456312293731945221911944028555688128175934421553211163334119551163911611894999108117970693288266504993837476332212657259793718339845962466531696690508758912601285019064089500296489453410381535756311193e14 ',
' -1.0414399480678916651564766907642884139887282626979191321264384754886370459889983293961404215384052283399731184308392476228871297713735944951644459310242466344126698328281692269561228176327036845566650027135201061836536623015104527502990360255694698258871800291026410179111571680418319563069024317375722e15 ',
' 4.2147709104951670858419264818832365210924767926381746712785245125225588262347029360528314681187063908373727723778210450391148039645303684199979441453560342313042051650782917945075116813119804636220732810723131054324025774116251126232324563609305452930844933140043717830313286746781861414517150006832914e15 ',
' -1.2531889521283083703459926155154706541943727300659016053924081290160592252062398123568872718614150497937742192982965748114850447585077502628097779255120303739666825738431362373083935498215775822297345358540080330082661762121237619010722015656376993662787297506264730316872558976090185175502681072941089e16 ',
' 2.5771356773742006602835032331605337646943021594213208618646413926629611746774953838029292397375396802872468020961010265196432887308994792700036703711078800864923615556946669150774500571017676979276107957040175757732243760156579502834159662530427371910887078710283067760058807061956812084033804092943772e16 ',
' -3.2707568867477651280446056304250432554533281768023905064427410597056686164760447280941805629068811791532963775048566588439783548744910230318597315995232148134307145693608562393517446239860994005819595102807190848663968078242982275000460603791317080477127622652115251392405847374698267692628455914339293e16 ',
' 1.928011728011168764688002408236594803044950771069760537345153193022098362158001133766661586364510290128892455530716787780762285926583233576631877726143871575582038686938288678264867916997755101706232708007749330610008537917636777663643554541096504239534839600581110446210280209495062951641169735500612e16 ',
]

for i, c in enumerate(values):
    c = RealField(1000)(c.replace(" ", ""))
    # print_double_double("", c)
    # print(RealField(53)(c))
    print("f64::from_bits(" + double_to_hex(c) + "),")

f64::from_bits(0x3fd9884533d4eede),
f64::from_bits(0xbfc32633ea493734),
f64::from_bits(0xbfa7efb20f01ec54),
f64::from_bits(0xbfa4fb95b2d5799e),
f64::from_bits(0xbfa904d8e7d5d790),
f64::from_bits(0xbfeae9499e97e1b7),
f64::from_bits(0x40472637a2ba1ff6),
f64::from_bits(0xc0a200beb7ea3ea8),
f64::from_bits(0x40f5f48b183d26ef),
f64::from_bits(0xc145648d6b8ddf8c),
f64::from_bits(0x4190c72b69beaf00),
f64::from_bits(0xc1d5480c24e2aa52),
f64::from_bits(0x4215ddcf12efdb80),
f64::from_bits(0xc2522ee4e2e71865),
f64::from_bits(0x428862578a03c5c7),
f64::from_bits(0xc2ba306e82491ba2),
f64::from_bits(0x42e64a5cbe99c9e4),
f64::from_bits(0xc30d99791146a19d),
f64::from_bits(0x432df29fa5dc037e),
f64::from_bits(0xc34642d7f3a3e806),
f64::from_bits(0x4356e3ba65783d6e),
f64::from_bits(0xc35d0cd713f40225),
f64::from_bits(0x43511fca876d5f12),


In [6]:
# Use arbitrary precision and special functions

# Set the target overflow limit for f64
f64_max = RealField(400)(2)**1024 * (RealField(400)(1) - RealField(400)(2**-53))  # == f64::MAX

# Use high precision for computations
R = RealField(400)

# Define the function to compare I0(x) with f64::MAX
def bessel_diff(x):
    return besseli(1, R(x)) - f64_max

# Use binary search to find the cutoff value where I0(x) ≈ f64::MAX
def find_cutoff(x_lo=700, x_hi=720, tol=1e-12):
    while x_hi - x_lo > tol:
        x_mid = (x_lo + x_hi) / 2
        if bessel_diff(x_mid) > 0:
            x_hi = x_mid
        else:
            x_lo = x_mid
    return (x_lo + x_hi) / 2

cutoff = find_cutoff()
print("Cutoff x where I0(x) ≈ f64::MAX is:", cutoff)
print("I0(cutoff) =", bessel_I(1, cutoff))


Cutoff x where I0(x) ≈ f64::MAX is: 713.9876098185423
I0(cutoff) = inf


In [26]:
from sage.all import *
from mpmath import mp, taylor

mp.prec = 500

def f(x):
    return mp.besseli(0, x) 

terms = 25
coeffs = taylor(f, mp.mpf(0), terms)

prec_bits = 350
RR = RealField(prec_bits)
L = LaurentSeriesRing(RealField(500), 'x', default_prec = 500)
x = L.gen()

series = L(0)
for n, c in enumerate(coeffs):
    series += RR(c) * x**(-n)

# Optional: print the series
print(series(subs=x))


2.59780277210771740096221707788581701134626033229944179534141469782417769044768017446002469666505676220698e-25*x^-24 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^-23 + 1.49633439673404522295423703686223059853544595140447847411665486594672634969786378048897422527907269503122e-22*x^-22 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^-21 + 7.24225848019277887909850725841319609691155840479767581472460955118215553253766069756663525035071184395112e-20*x^-20 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^-19 + 2.89690339207711155163940290336527843876462336191907032588984382047286221301506427902665410014028473758045e-17*x^-18 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^-17 + 9.3859669903298414273116654069035021415973796926177878558830